## All Imports

In [9]:
import requests
import markdown
from bs4 import BeautifulSoup

import google.generativeai as genai
from google.genai import types

import re
import wave

import os

In [6]:
from google.colab import userdata
FIRECRAWL_API = userdata.get('FIRECRAWL_API')

from google.colab import userdata
GEMINI_API = userdata.get('GEMINI_KEY')

# Data Scarpping using ***Firecrawl API***

In [7]:
URL_TO_SCRAPE = "https://www.galactanet.com/oneoff/theegg_mod.html"
url = "https://api.firecrawl.dev/v1/scrape"

def content():
    """ Firecrawl is used to scrap the provided site using it's automation tools """
    response = requests.request("POST",url, headers={"Authorization": f"Bearer {FIRECRAWL_API}"},
        json={
            "url": URL_TO_SCRAPE,
            "formats": ["markdown"],
            "onlyMainContent": True
        }
    )
    data = response.json()
    # print(data)

    markdown_content = data["data"]["markdown"]
    content = markdown.markdown(markdown_content)
    soupDetails = BeautifulSoup(markup = content, features = "html.parser")

    """ For Simulation Thoery Podcast """
    # simulation_theory = soupDetails.get_text(separator="\n")

    # lines = simulation_theory.splitlines()
    # filtered_lines = [line for line in lines if not line.startswith("Image:") and "UPDATED BY" not in line]
    # simulation_theory_filtered = "\n".join(filtered_lines)

    # return simulation_theory_filtered[85:14200]


    """ For Egg Theory Podcast """
    the_egg_theory = soupDetails.get_text(separator="\n")

    lines = the_egg_theory.splitlines()
    filtered_lines = [line for line in lines if not line.startswith("Image:") and "UPDATED BY" not in line]
    the_egg_theory_filtered = "\n".join(filtered_lines)

    The_Egg_Theory = ""
    The_Egg_Theory += the_egg_theory_filtered[800:]

    response = requests.request("POST",url, headers={"Authorization": f"Bearer {FIRECRAWL_API}"},
        json={
            "url": "https://www.hopelesslyellow.com/winter-2022/i-love-sushi-but-hate-to-eat-it-89d7b",
            "formats": ["markdown"],
            "onlyMainContent": True
        }
    )
    data = response.json()
    markdown_content = data["data"]["markdown"]
    content = markdown.markdown(markdown_content)
    soupDetails = BeautifulSoup(markup = content, features = "html.parser")
    the_egg_theory_2 = soupDetails.get_text(separator="\n")
    lines = the_egg_theory_2.splitlines()
    filtered_lines = [line for line in lines if not line.startswith("Image:") and "UPDATED BY" not in line]
    the_egg_theory_2_filtered = "\n".join(filtered_lines)

    The_Egg_Theory += the_egg_theory_2_filtered[60:3350]
    return The_Egg_Theory

print(content())



The Egg


By: Andy Weir


You were on your way home when you
died.


It was a car accident. Nothing
particularly remarkable, but fatal nonetheless. You left behind a wife and two
children. It was a painless death. The EMTs tried
their best to save you, but to no avail. Your body was so utterly shattered you
were better off, trust me.


And that’s when you met me.


“What… what happened?” You asked.
“Where am I?”


“You died,” I said,
matter-of-factly. No point in mincing words.


“There was a… a truck and it was
skidding…”


“Yup,” I said.


“I… I died?”


“Yup. But don’t feel bad about it.
Everyone dies,” I said.


You looked around. There was
nothingness. Just you and me. “What is this place?” You asked. “Is this the
afterlife?”


“More or less,” I said.


“Are you god?” You asked.


“Yup,” I replied. “I’m God.”


“My kids… my wife,” you said.


“What about them?”


“Will they be all right?”


“That’s what I like to see,” I
said. “You just died and your main concern is for your fam

# Script Generation using ***Gemini Multimodel System***

In [10]:
clean_text = content() #Calling content() to obtained the scrapped data.

prompt = f"""
Summarize the following article in simple Englishand convert it into podcast conversation between two hosts: Alex (curious) and Aria (expert).
Make it engaging and educational. Make Alex (curious) a host and Aria (expert) a guest. Guest should be an expert in the topic of the article.
Guest name is only Aria (no last name).
The Podcast name is VoxGem (i.e., VoxGem Podcast).
Let Alex ask critical and engaging questiona and Aria answer them all gracefully and smartly.
Give the podcast a title and a short description.
Give the podcast a catchy introduction and a conclusion.
Give no trace of the original article in the podcast script and AI should not mention the article.
Add your own creative touch to the podcast script.
Add own content to the podcast script.
Make sure the podcast script is engaging and educational.
Add pauses in the script where appropriate.
Make sure the script is in a conversational tone.
Make the script so it can sound natural with two speakers talking and having breaks or pauses where needed.
Make the podcast atleast of 10 mintues and max of 15 mintues.
Add laughs and emotions to the script where appropriate.

Here is the content to base the conversation on:
\"\"\"
{clean_text}
\"\"\"
"""
genai.configure(api_key=GEMINI_API)
model = genai.GenerativeModel("gemini-1.5-pro")

""" Gemini-1.5.-pro version is used for text generation as per the provided prompt to create a auotmated Script for our AI Voices models
to use and perform the podcast Series. """
response = model.generate_content(prompt)
podcast_script = response.text
print("\nVoxGem Podcast Script:\n")
print(podcast_script)

with open("voxgem_script.txt", "w", encoding="utf-8") as f:
    f.write(podcast_script)


VoxGem Podcast Script:

## VoxGem Podcast: Cracking the Cosmic Egg: Are We All Just One?

**Description:** Join Alex, your curious host, as he delves into the mind-bending Egg Theory with special guest, Aria, an expert on consciousness and metaphysics.  They explore the fascinating idea that every human who ever lived, and ever will live, is just a different version of the same soul – you! Prepare to question everything you thought you knew about life, the universe, and everything.

**(Intro Music with a futuristic, ethereal vibe)**

**Alex:** Welcome back to VoxGem, where we explore the gems of knowledge hidden within the vast cosmos of thought. Today, we're cracking open a particularly intriguing concept – The Egg Theory. And to help us understand this mind-boggling idea, we have the brilliant Aria joining us. Welcome to the show, Aria!

**Aria:** Thanks for having me, Alex. Excited to dive into this!

**Alex:**  So, Aria, for our listeners who are hearing about this for the first t

# ***AI-Podcast Creation !!***

In [11]:
""" All directly refereneced from the Gemini Multimodel Documentation : https://ai.google.dev/gemini-api/docs/speech-generation
- Used Gemini-2.5-flash-preview-tts model for speech generation for multiagent interaction between two voices.
- Uses the previously generated voxgem_script to create the Podcast.
"""
from google import genai

client = genai.Client(api_key=GEMINI_API)

def wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
   with wave.open(filename, "wb") as wf:
      wf.setnchannels(channels)
      wf.setsampwidth(sample_width)
      wf.setframerate(rate)
      wf.writeframes(pcm)

with open("voxgem_script.txt", "r", encoding="utf-8") as f:
    full_script = f.read()

def parse_script(script_text):
    pattern = r"(Alex|Aria):(.+?)(?=(Alex|Aria):|\Z)"
    matches = re.findall(pattern, script_text, re.DOTALL)
    parts = []
    for speaker, text, _ in matches:
        line = f"{speaker.strip()}: {text.strip()}"
        parts.append({"text": line})
    return parts

conversation_parts = parse_script(full_script)

response = client.models.generate_content(
   model="gemini-2.5-flash-preview-tts",
   contents=conversation_parts,
   config=types.GenerateContentConfig(
      response_modalities=["AUDIO"],
      speech_config=types.SpeechConfig(
         multi_speaker_voice_config=types.MultiSpeakerVoiceConfig(
            speaker_voice_configs=[
               types.SpeakerVoiceConfig(
                  speaker="Alex",
                    voice_config=types.VoiceConfig(
                        prebuilt_voice_config=types.PrebuiltVoiceConfig(
                            voice_name="Achird"
                        )
                    )
                ),
               types.SpeakerVoiceConfig(
                  speaker="Aria",
                    voice_config=types.VoiceConfig(
                        prebuilt_voice_config=types.PrebuiltVoiceConfig(
                            voice_name="Aoede"
                        )
                    )
                ),
            ]
         )
      )
   )
)

data = response.candidates[0].content.parts[0].inline_data.data

file_name = 'voxgem_podcast.wav'
wave_file(file_name, data)

print("🎧 VoxGem podcast audio saved as 'voxgem_podcast.wav'")

🎧 VoxGem podcast audio saved as 'voxgem_podcast.wav'
